In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.metrics import classification_report


data_dir = "/content/drive/MyDrive/archive (13)/OA/TRAIN"
img_height, img_width = 224, 224
batch_size = 32

train_ds = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
).flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="training"
)

val_ds = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2
).flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    subset="validation",
    shuffle=False
)


test_ds = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0/255).flow_from_directory(
    "/content/drive/MyDrive/archive (13)/OA/TEST",
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(img_height, img_width, 3)
)


base_model.trainable = False


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(train_ds.num_classes, activation='softmax')(x)


model = Model(inputs=base_model.input, outputs=predictions)


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)


lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=3,
    verbose=1,
    min_lr=1e-6
)

early_stopping = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

# Train the Model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    steps_per_epoch=len(train_ds),
    validation_steps=len(val_ds),
    callbacks=[lr_scheduler, early_stopping]
)

# Save the Model
model.save("imagenet_finetuned_model.keras")

# Evaluate the Model on Test Data
test_loss, test_accuracy = model.evaluate(test_ds, steps=len(test_ds))
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

# Classification Report
y_true = test_ds.classes
y_pred = model.predict(test_ds, steps=len(test_ds))
y_pred_classes = y_pred.argmax(axis=1)

print(classification_report(y_true, y_pred_classes, target_names=test_ds.class_indices.keys()))


Found 1198 images belonging to 2 classes.
Found 298 images belonging to 2 classes.
Found 408 images belonging to 2 classes.
Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


38/38 ━━━━━━━━━━━━━━━━━━━━ 29s 463ms/step - accuracy: 0.8112 - loss: 0.4466 - val_accuracy: 0.8221 - val_loss: 0.5220 - learning_rate: 0.0010
Epoch 2/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


38/38 ━━━━━━━━━━━━━━━━━━━━ 4s 103ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/20


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/callback_list.py:96: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate.
  callback.on_epoch_end(epoch, logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss,learning_rate
  current = self.get_monitor_value(logs)


38/38 ━━━━━━━━━━━━━━━━━━━━ 7s 163ms/step - accuracy: 0.9617 - loss: 0.0995 - val_accuracy: 0.8087 - val_loss: 0.5560 - learning_rate: 0.0010
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 460us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 161ms/step - accuracy: 0.9809 - loss: 0.0589 - val_accuracy: 0.8221 - val_loss: 0.5689 - learning_rate: 0.0010
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 189ms/step - accuracy: 0.9745 - loss: 0.0657 - val_accuracy: 0.8356 - val_loss: 0.5154 - learning_rate: 0.0010
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 558us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - learning_rate: 0.0010
Epoch 9/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 164ms/step - accuracy: 0.9840 - loss: 0.0391 - val_accuracy: 0.8523 - val_loss: 0.5828 - learning_rate: 0.0010
Epoch 10/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 510us/

In [8]:
def predict_image(image_path, model, class_indices):

    # Load and preprocess the image
    img = load_img(image_path, target_size=(img_height, img_width))
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # To Generate predictions
    predictions = model.predict(img_array)
    predicted_class_index = np.argmax(predictions[0])
    confidence_score = predictions[0][predicted_class_index] * 100  # Convert to percentage

    # Map index to class label
    class_labels = {v: k for k, v in class_indices.items()}
    predicted_label = class_labels[predicted_class_index]

    return predicted_label, confidence_score

image_path = "/content/drive/MyDrive/archive (13)/OA/TEST/Caries/202_2.jpg"
predicted_label, confidence_score = predict_image(image_path, model, train_ds.class_indices)

print(f"The predicted class for the image is: {predicted_label}")
print(f"Confidence score: {confidence_score:.2f}%")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
The predicted class for the image is: Caries
Confidence score: 99.62%
